# Dicionário de variáveis:
- ID number
- Diagnosis (M = malignant, B = benign)
- radius (mean of distances from center to points on the perimeter)
- texture (standard deviation of gray-scale values)
- perimeter
- area
- smoothness (local variation in radius lengths)
- compactness (perimeter^2 / area - 1.0)
- concavity (severity of concave portions of the contour)
- concave points (number of concave portions of the contour)
- symmetry
- fractal dimension ("coastline approximation" - 1)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import uniform, randint
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [3]:
!pip install catboost

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [5]:
df = pd.read_csv("/content/drive/MyDrive/Udemy/ML com Python/Datasets/data_cancer2.csv")

In [6]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [7]:
df.dtypes

,0
id,int64
diagnosis,object
radius_mean,float64
texture_mean,float64
perimeter_mean,float64
area_mean,float64
smoothness_mean,float64
compactness_mean,float64
concavity_mean,float64
concave points_mean,float64


In [8]:
df.shape

(569, 33)

In [9]:
df.isnull().sum()

,0
id,0
diagnosis,0
radius_mean,0
texture_mean,0
perimeter_mean,0
area_mean,0
smoothness_mean,0
compactness_mean,0
concavity_mean,0
concave points_mean,0


In [10]:
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)

In [11]:
df.head(1)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.8,1001.0,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189


In [12]:
df.describe()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,0.405172,1.216853,2.866059,40.337079,0.007041,0.025478,0.031894,0.011796,0.020542,0.003795,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,0.277313,0.551648,2.021855,45.491006,0.003003,0.017908,0.030186,0.006170,0.008266,0.002646,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,0.111500,0.360200,0.757000,6.802000,0.001713,0.002252,0.000000,0.000000,0.007882,0.000895,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,0.232400,0.833900,1.606000,17.850000,0.005169,0.013080,0.015090,0.007638,0.015160,0.002248,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,0.324200,1.108000,2.287000,24.530000,0.006380,0.020450,0.025890,0.010930,0.018730,0.003187,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,0.478900,1.474000,3.357000,45.190000,0.008146,0.032450,0.042050,0.014710,0.023480,0.004558,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,2.873000,4.885000,21.980000,542.200000,0.031130,0.135400,0.396000,0.052790,0.078950,0.029840,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [13]:
# Transformar a variavel "diagnosis" em numerica
df['diagnosis'].replace({'M': 1, 'B': 0}, inplace=True)

In [14]:
df.head(1)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.8,1001.0,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189


In [15]:
df.describe()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,0.372583,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,0.405172,1.216853,2.866059,40.337079,0.007041,0.025478,0.031894,0.011796,0.020542,0.003795,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,0.483918,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,0.277313,0.551648,2.021855,45.491006,0.003003,0.017908,0.030186,0.006170,0.008266,0.002646,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,0.000000,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,0.111500,0.360200,0.757000,6.802000,0.001713,0.002252,0.000000,0.000000,0.007882,0.000895,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,0.000000,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,0.232400,0.833900,1.606000,17.850000,0.005169,0.013080,0.015090,0.007638,0.015160,0.002248,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,0.000000,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,0.324200,1.108000,2.287000,24.530000,0.006380,0.020450,0.025890,0.010930,0.018730,0.003187,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,1.000000,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,0.478900,1.474000,3.357000,45.190000,0.008146,0.032450,0.042050,0.014710,0.023480,0.004558,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,1.000000,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,2.873000,4.885000,21.980000,542.200000,0.031130,0.135400,0.396000,0.052790,0.078950,0.029840,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


## Separacao das Variaveis

In [16]:
X = df.drop(columns=["diagnosis"])
y = df["diagnosis"]

## Divisao entre treino e teste

In [17]:
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=.2, random_state=42)

## Parametros que vou testar

In [22]:
param_grids = {
    "Logistic Regression": {
        "model__C": uniform(0.01, 10),  # Regularização
        "model__penalty": ["l2", "none"]
    },
    "Naive Bayes": {
    },
    "Decision Tree": {
        "model__max_depth": randint(1, 20),
        "model__min_samples_split": randint(2, 10)
    },
    "Random Forest": {
        "model__n_estimators": randint(10, 200),
        "model__max_depth": randint(1, 20),
        "model__min_samples_split": randint(2, 10)
    },
    "Bagging": {
        "model__n_estimators": randint(10, 200),
        "model__max_samples": uniform(0.5, 1.0)
    },
    "Extra Tree": {
        "model__max_depth": randint(1, 20),
        "model__min_samples_split": randint(2, 10),
        "model__criterion": ["gini", "entropy"]
    },
    "AdaBoost": {
        "model__n_estimators": randint(10, 200),
        "model__learning_rate": uniform(0.01, 2)
    },
    "Gradient Boosting": {
        "model__n_estimators": randint(10, 200),
        "model__learning_rate": uniform(0.01, 2),
        "model__max_depth": randint(1, 10)
    },
    "XGBoost": {
        "model__n_estimators": randint(10, 200),
        "model__learning_rate": uniform(0.01, 2),
        "model__max_depth": randint(1, 10)
    },
    "LightGBM": {
        "model__n_estimators": randint(10, 200),
        "model__learning_rate": uniform(0.01, 2),
        "model__num_leaves": randint(10, 50)
    },
    "CatBoost": {
        "model__iterations": randint(10, 200),
        "model__learning_rate": uniform(0.01, 2),
        "model__depth": randint(1, 10)
    }
}

## Preprocessamento com Pipeline

In [23]:
# Padronizacao
standard_scaler = StandardScaler()

# Nenhum preprocessamento
sem_preprocess = 'passthrough'

In [24]:
# Criacao dos pipelines

pipes = {
    'Logistic Regression': Pipeline([
        ('scaler', standard_scaler),
        ('model', LogisticRegression())
    ]),
    'Naive Bayes': Pipeline([
        ('scaler', standard_scaler),
        ('model', GaussianNB())
    ]),
    'Decision Tree': Pipeline([
        ('scaler', sem_preprocess),
        ('model', DecisionTreeClassifier())
    ]),
    'Random Forest': Pipeline([
        ('scaler', sem_preprocess),
        ('model', RandomForestClassifier())
    ]),
    'Bagging': Pipeline([
        ('scaler', sem_preprocess),
        ('model', BaggingClassifier())
    ]),
    'Extra Tree': Pipeline([
        ('scaler', sem_preprocess),
        ('model', ExtraTreesClassifier())
    ]),
    'AdaBoost': Pipeline([
        ('scaler', standard_scaler),
        ('model', AdaBoostClassifier())
    ]),
    'Gradient Boosting': Pipeline([
        ('scaler', standard_scaler),
        ('model', GradientBoostingClassifier())
    ]),
    'XGBoost': Pipeline([
        ('scaler', standard_scaler),
        ('model', XGBClassifier(eval_metric='logloss', use_label_encoder=False))
    ]),
    'LightGBM': Pipeline([
        ('scaler', standard_scaler),
        ('model', LGBMClassifier())
    ]),
    'CatBoost': Pipeline([
        ('scaler', standard_scaler),
        ('model', CatBoostClassifier(verbose=0))
    ])
}

## Aplicacao da RandomizedSearchCV

In [28]:
best_models = {}

for model_name, pipe in pipes.items():
  print(f"Treinando {model_name}")
  param_grid = param_grids.get(model_name, {})
  search = RandomizedSearchCV(
      pipe,
      param_distributions=param_grid,
      n_iter=50,
      scoring='accuracy',
      cv=5,
      random_state=42,
      n_jobs=-1
  )
  search.fit(X_tr, y_tr)
  best_models[model_name] = search
  print(f"Melhor acuracia: {search.best_score_}")
  print(f"Melhor hiperparametros: {search.best_params_}")

Treinando Logistic Regression
Melhor acuracia: 0.9780219780219781
Melhor hiperparametros: {'model__C': 3.7554011884736247, 'model__penalty': 'l2'}
Treinando Naive Bayes
Melhor acuracia: 0.9340659340659341
Melhor hiperparametros: {}
Treinando Decision Tree
Melhor acuracia: 0.9318681318681319
Melhor hiperparametros: {'model__max_depth': 17, 'model__min_samples_split': 5}
Treinando Random Forest
Melhor acuracia: 0.9626373626373625
Melhor hiperparametros: {'model__max_depth': 7, 'model__min_samples_split': 3, 'model__n_estimators': 84}
Treinando Bagging
Melhor acuracia: 0.956043956043956
Melhor hiperparametros: {'model__max_samples': 0.9458327528535911, 'model__n_estimators': 84}
Treinando Extra Tree
Melhor acuracia: 0.9692307692307693
Melhor hiperparametros: {'model__criterion': 'entropy', 'model__max_depth': 11, 'model__min_samples_split': 4}
Treinando AdaBoost
Melhor acuracia: 0.9802197802197803
Melhor hiperparametros: {'model__learning_rate': 1.520722820635305, 'model__n_estimators': 1

## Avaliacao de Resultados

In [30]:
for model_name, search in best_models.items():
    print(f"\nModelo: {model_name}")
    print(f"Melhores hiperparâmetros: {search.best_params_}")
    best_model = search.best_estimator_
    y_pred = best_model.predict(X_ts)
    acc = accuracy_score(y_ts, y_pred)
    print(f"Acurácia: {acc:.2f}")


Modelo: Logistic Regression
Melhores hiperparâmetros: {'model__C': 3.7554011884736247, 'model__penalty': 'l2'}
Acurácia: 0.97

Modelo: Naive Bayes
Melhores hiperparâmetros: {}
Acurácia: 0.96

Modelo: Decision Tree
Melhores hiperparâmetros: {'model__max_depth': 17, 'model__min_samples_split': 5}
Acurácia: 0.93

Modelo: Random Forest
Melhores hiperparâmetros: {'model__max_depth': 7, 'model__min_samples_split': 3, 'model__n_estimators': 84}
Acurácia: 0.96

Modelo: Bagging
Melhores hiperparâmetros: {'model__max_samples': 0.9458327528535911, 'model__n_estimators': 84}
Acurácia: 0.96

Modelo: Extra Tree
Melhores hiperparâmetros: {'model__criterion': 'entropy', 'model__max_depth': 11, 'model__min_samples_split': 4}
Acurácia: 0.97

Modelo: AdaBoost
Melhores hiperparâmetros: {'model__learning_rate': 1.520722820635305, 'model__n_estimators': 143}
Acurácia: 0.96

Modelo: Gradient Boosting
Melhores hiperparâmetros: {'model__learning_rate': 0.5924582803960838, 'model__max_depth': 3, 'model__n_esti